In [200]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB  
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix  
import matplotlib.pyplot as plt
import regex as re

In [201]:
data = pd.read_csv("sms_spam.csv")
data.columns.values[0] = "Label"
data.columns.values[1] = "SMS"
data

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5569,spam,This is the 2nd time we have tried 2 contact u...
5570,ham,Will ü b going to esplanade fr home?
5571,ham,"Pity, * was in mood for that. So...any other s..."
5572,ham,The guy did some bitching but I acted like i'd...


In [202]:
data['SMS'].value_counts(normalize=True)

SMS
Sorry, I'll call later                                                                                                                                      0.005382
I cant pick the phone right now. Pls send a message                                                                                                         0.002153
Ok...                                                                                                                                                       0.001794
Ok                                                                                                                                                          0.000718
Ok.                                                                                                                                                         0.000718
                                                                                                                                                              ...   
I gott

In [203]:
data_randomized = data.sample(frac=1, random_state=1)
data_randomized

,Label,SMS
1447,ham,Looks like u wil b getting a headstart im leav...
2032,ham,"I noe la... U wana pei bf oso rite... K lor, o..."
4432,ham,2mro i am not coming to gym machan. Goodnight.
4888,spam,Todays Vodafone numbers ending with 4882 are s...
5276,ham,"Hi. Hope ur day * good! Back from walk, table ..."
...,...,...
905,ham,"We're all getting worried over here, derek and..."
5192,spam,Our records indicate u maybe entitled to 5000 ...
3980,ham,CERI U REBEL! SWEET DREAMZ ME LITTLE BUDDY!! C...
235,spam,Text & meet someone sexy today. U can find a d...


In [204]:
'''
Here we can't use train_test_split as the data is not processed. There are white spaces, punctuation
marks and so on. We need a bag of words to work on.
'''
training_test_index = round(len(data_randomized)*0.8)
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)
training_set.shape, test_set.shape

((4459, 2), (1115, 2))

In [205]:
training_set.head()

,Label,SMS
0,ham,Looks like u wil b getting a headstart im leav...
1,ham,"I noe la... U wana pei bf oso rite... K lor, o..."
2,ham,2mro i am not coming to gym machan. Goodnight.
3,spam,Todays Vodafone numbers ending with 4882 are s...
4,ham,"Hi. Hope ur day * good! Back from walk, table ..."


In [206]:
training_set['SMS'] = training_set['SMS'].str.replace(r'\W', ' ', regex=True)# Removes punctuation
training_set['SMS'] = training_set['SMS'].str.lower()
training_set['SMS'][1]

'i noe la    u wana pei bf oso rite    k lor  other days den   '

In [207]:
training_set['SMS'] = training_set['SMS'].str.split()

In [208]:
vocabulary = set()
for text in training_set['SMS']:
   for word in text:
      vocabulary.add(word)

vocabulary = list(vocabulary)
len(vocabulary)
if "text" in vocabulary:
   print("yes")

yes


In [209]:
word_counts_per_text = {'secret': [2,1,1],
                       'prize': [2,0,1],
                       'claim': [1,0,1],
                       'now': [1,0,1],
                       'coming': [0,1,0],
                       'to': [0,1,0],
                       'my': [0,1,0],
                       'party': [0,1,0],
                       'winner': [0,0,1]
                      }
word_counts = pd.DataFrame(word_counts_per_text)
word_counts.head()

,secret,prize,claim,now,coming,to,my,party,winner
0,2,2,1,1,0,0,0,0,0
1,1,0,0,0,1,1,1,1,0
2,1,1,1,1,0,0,0,0,1


In [210]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
   for word in sms:
      word_counts_per_sms[word][index] += 1
    
word_counts_per_sms.keys()

dict_keys(['nimya', 'evng', '3hrs', 'laundry', 'pouch', 'traffic', 'rum', 'deliveredtomorrow', 'family', 'practical', 'blackberry', 'sacked', 'one', 'boss', 'accidentally', 'y', 'clas', 'mahfuuz', 'raining', 'painful', 'waaaat', 'mornin', '07973788240', 'curfew', 'showr', 'vivek', 'howz', 'quoting', 'ahmad', 'tomorro', '140', 'respect', 'coulda', 'elaine', 'ten', '08719839835', 'well', 'sittin', 'moon', 'asked', 'ma', 'ebay', 'farrell', 'wasted', '86888', 'scool', 'genuine', 'talents', 'coat', 'option', 'thriller', '08714712412', 'nobody', 'hr', 'honeymoon', 'donate', 'taxi', 'making', 'assume', 'itna', 'messy', 'andre', 'getsleep', 'mila', 'sunroof', 'pobox75ldns7', 'iam', 'nt', 'whenever', 'jokin', 'sextextuk', 'shoulders', 'vettam', 'chatting', 'wall', 'wither', 'quiet', 'evry1', 'nalla', 'weiyi', 'officer', 'standing', 'yoyyooo', 'dodda', 'popped', 'checkup', 'expired', 'brum', 'rule', 'gower', 'three', 'away', '09050090044', 'work', 'until', 'nan', 'shakespeare', 'tata', '8883', '

1. Remove punctuations
2. Create vocabulary
3. Do train test split
4. Feed to NB classifier
5. Share the accuracy

In [211]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,nimya,evng,3hrs,laundry,pouch,traffic,rum,deliveredtomorrow,family,practical,...,08708034412,realised,horny,couldn,mention,improved,mobypobox734ls27yf,successful,077xxx,hardcore
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [212]:
training_set_clean = pd.concat([training_set, word_counts], axis=1)
training_set_clean.head()

,Label,SMS,nimya,evng,3hrs,laundry,pouch,traffic,rum,deliveredtomorrow,...,08708034412,realised,horny,couldn,mention,improved,mobypobox734ls27yf,successful,077xxx,hardcore
0,ham,"[looks, like, u, wil, b, getting, a, headstart...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[i, noe, la, u, wana, pei, bf, oso, rite, k, l...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[2mro, i, am, not, coming, to, gym, machan, go...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,spam,"[todays, vodafone, numbers, ending, with, 4882...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[hi, hope, ur, day, good, back, from, walk, ta...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [213]:
spam_messages = training_set_clean[training_set_clean['Label'] == 'spam']
ham_messages = training_set_clean[training_set_clean['Label'] == 'ham']
spam_messages.head()

,Label,SMS,nimya,evng,3hrs,laundry,pouch,traffic,rum,deliveredtomorrow,...,08708034412,realised,horny,couldn,mention,improved,mobypobox734ls27yf,successful,077xxx,hardcore
3,spam,"[todays, vodafone, numbers, ending, with, 4882...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,spam,"[we, tried, to, contact, you, re, our, offer, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,spam,"[freemsg, why, haven, t, you, replied, to, my,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25,spam,"[loans, for, any, purpose, even, if, you, have...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
47,spam,"[would, you, like, to, see, my, xxx, pics, the...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [214]:
# P(Spam) and P(Ham)
p_spam = len(spam_messages) / len(training_set_clean)
p_ham = len(ham_messages) / len(training_set_clean)
p_spam, p_ham

(0.1347835837631756, 0.8652164162368244)

In [215]:
# N_Ham
n_words_per_ham_message = ham_messages['SMS'].apply(len)
n_ham = n_words_per_ham_message.sum()
n_words_per_ham_message, n_ham

(0       34
 1       14
 2        9
 4       20
 5       15
         ..
 4453    24
 4454     6
 4455    17
 4456    10
 4458    27
 Name: SMS, Length: 3858, dtype: int64,
 57367)

In [216]:
# N_Spam
n_words_per_spam_message = spam_messages['SMS'].apply(len)
n_spam = n_words_per_spam_message.sum()
n_words_per_spam_message, n_spam
print(n_spam)

15239


In [217]:
# N_Vocabulary
n_vocabulary = len(vocabulary)

# Laplace smoothing
alpha = np.int64(1)
n_vocabulary, alpha

(7802, 1)

In [218]:
# Initiate parameters
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}

# Calculate parameters
for word in vocabulary:
   n_word_given_spam = spam_messages[word].sum() # spam_messages already defined
   p_word_given_spam = (n_word_given_spam + alpha)/ (alpha * n_vocabulary + n_spam)
   parameters_spam[word] = p_word_given_spam

   n_word_given_ham = ham_messages[word].sum() # ham_messages already defined
   p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
   parameters_ham[word] = p_word_given_ham
parameters_spam

{'nimya': 4.34008940584176e-05,
 'evng': 4.34008940584176e-05,
 '3hrs': 8.68017881168352e-05,
 'laundry': 4.34008940584176e-05,
 'pouch': 4.34008940584176e-05,
 'traffic': 4.34008940584176e-05,
 'rum': 4.34008940584176e-05,
 'deliveredtomorrow': 0.0001736035762336704,
 'family': 4.34008940584176e-05,
 'practical': 4.34008940584176e-05,
 'blackberry': 4.34008940584176e-05,
 'sacked': 4.34008940584176e-05,
 'one': 0.0003038062584089232,
 'boss': 4.34008940584176e-05,
 'accidentally': 4.34008940584176e-05,
 'y': 4.34008940584176e-05,
 'clas': 4.34008940584176e-05,
 'mahfuuz': 4.34008940584176e-05,
 'raining': 4.34008940584176e-05,
 'painful': 4.34008940584176e-05,
 'waaaat': 4.34008940584176e-05,
 'mornin': 4.34008940584176e-05,
 '07973788240': 8.68017881168352e-05,
 'curfew': 4.34008940584176e-05,
 'showr': 4.34008940584176e-05,
 'vivek': 4.34008940584176e-05,
 'howz': 4.34008940584176e-05,
 'quoting': 0.0001736035762336704,
 'ahmad': 4.34008940584176e-05,
 'tomorro': 4.34008940584176e-0

In [219]:
import re

def classify(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham: 
         p_ham_given_message *= parameters_ham[word]

   print('P(Spam|message):', p_spam_given_message)
   print('P(Ham|message):', p_ham_given_message)

   if p_ham_given_message > p_spam_given_message:
      print('Label: Ham')
   elif p_ham_given_message < p_spam_given_message:
      print('Label: Spam')
   else:
      print('Equal proabilities, have a human classify this!')

In [220]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.3223124637479498e-25
P(Ham|message): 1.9256070327302398e-27
Label: Spam


In [221]:
def classify_test_set(message):
   '''
   message: a string
   '''

   message = re.sub('\W', ' ', message)
   message = message.lower().split()

   p_spam_given_message = p_spam
   p_ham_given_message = p_ham

   for word in message:
      if word in parameters_spam:
         p_spam_given_message *= parameters_spam[word]

      if word in parameters_ham:
         p_ham_given_message *= parameters_ham[word]

   if p_ham_given_message > p_spam_given_message:
      return 'ham'
   elif p_spam_given_message > p_ham_given_message:
      return 'spam'
   else:
      return 'needs human classification'

In [222]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
print(test_set.head())
correct = 0
total = test_set.shape[0]

for row in test_set.iterrows():
   row = row[1]
   if row['Label'] == row['predicted']:
      correct += 1

print('Correct:', correct)
print('Incorrect:', total - correct)
print('Accuracy:', correct/total * 100)

  Label                                                SMS predicted
0   ham                           Wherre's my boytoy ? :-(       ham
1   ham          Later i guess. I needa do mcat study too.       ham
2   ham             But i haf enuff space got like 4 mb...       ham
3  spam  Had your mobile 10 mths? Update to latest Oran...      spam
4   ham  All sounds good. Fingers . Makes it difficult ...       ham
Correct: 1103
Incorrect: 12
Accuracy: 98.9237668161435
